In [ ]:
!pip install dtreeviz

In [ ]:
#Import all required Libraries

import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt

import seaborn as sns
import os
import numpy as np
import glob

import re
import warnings
warnings.filterwarnings('ignore')

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.model_selection import GridSearchCV, cross_val_score, KFold, RepeatedStratifiedKFold

from pywt import wavedec

from random import sample
from numpy.random import uniform
from math import isnan

from dtreeviz.trees import *

from IPython.core.display import HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
data = pd.read_csv('../input/tabular-playground-series-jun-2022/data.csv', index_col='row_id')


In [ ]:
sub = pd.read_csv('../input/tabular-playground-series-jun-2022/sample_submission.csv')

In [ ]:
data.info()

In [ ]:
# memory reduce
for col in data.columns:
    if data[col].dtype == "float64":
        data[col]=pd.to_numeric(data[col], downcast="float")
    if data[col].dtype == "int64":
        data[col]=pd.to_numeric(data[col], downcast="integer")

In [ ]:
data.info()

In [ ]:
data.isna().sum()

In [ ]:
corr = data.corr()

In [ ]:
fig, axes= plt.subplots(ncols=1, nrows=1, figsize=(30, 15))
sns.heatmap(corr, ax=axes, vmin=-1, vmax=1)

In [ ]:
col = data.columns

# fig, axes= plt.subplots(ncols=4, nrows=20, figsize=(25, 35))

# for i, ax in zip(range(79), axes.flat):
#     plot = sns.distplot(x=data[col[i]].sample(frac=0.05), ax=ax)
#     plot.set_title(col[i+1])
#     fig.tight_layout()
# plt.show()

In [ ]:
fig, axes= plt.subplots(ncols=1, nrows=1, figsize=(30, 15))
sns.heatmap(data.isnull())

In [ ]:
fig, axes= plt.subplots(ncols=1, nrows=1, figsize=(30, 15))
sns.boxplot(data=data)

In [ ]:
# Train Test Split
col_notmiss = [x for x in col if x.startswith("F_2")]
col_miss = [x for x in col if x not in col_notmiss]

col_miss_corr = [x for x in col_miss if x.startswith("F_4")] 
col_miss_notcorr = [x for x in col_miss if x not in col_miss_corr]

data_miss_notcorr = data[col_miss_notcorr]
data_miss_corr = data[col_miss_corr]

#data_train = data_missing.iloc[0:700000]
#data_test = data_missing.iloc[700000:]

In [ ]:
print(data_miss_notcorr.shape, data_miss_corr.shape)

In [ ]:
# Importing the SimpleImputer class
from sklearn.impute import SimpleImputer
 
# Imputer object using the mean strategy and missing_values type for imputation
imputer = SimpleImputer(missing_values = np.nan,
                        strategy ='median')
 
# Fitting the data to the imputer object
imputer = imputer.fit(data_miss_notcorr)
 
# Imputing the data    
data = imputer.transform(data_miss_notcorr)

In [ ]:
from tqdm.notebook import tqdm

import lightgbm
import xgboost
#import catboost

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

SEED = 622

In [ ]:
# code from: https://www.kaggle.com/code/hiro5299834/tps-jun-2022-iterativeimputer-baseline

xgb = xgboost.XGBRegressor(
        n_estimators=500,
        random_state=SEED,
        tree_method='gpu_hist',
    )
imp = IterativeImputer(
    estimator=xgb,
    missing_values=np.nan,
    max_iter=10,
    initial_strategy='median',
    imputation_order='ascending',
    verbose=2,
    random_state=SEED
)

data_2 = imp.fit_transform(data_miss_corr)

In [ ]:
data = pd.DataFrame(data,columns=col_miss_notcorr)

In [ ]:
data_2 = pd.DataFrame(data_2,columns=col_miss_corr)

In [ ]:
result = pd.concat([data, data_2], axis=1, join='inner')
result

In [ ]:
#iterating through a NumPy array is much faster than using df.iterrows() !
for i, row in enumerate(sub.values):
    row_col = row[0]
    imputed_row = row_col.split("-")[0] #get the row index
    imputed_col = row_col.split("-")[1] #get the column index
    sub.at[i, "value"] = result.iloc[int(imputed_row)][imputed_col]

In [ ]:
sub.head()

In [ ]:
sub.to_csv("submission.csv", index=False)